# sk_learn Outlier removal

In [1656]:
import pandas as pd;
import numpy as np;
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [1657]:
### CUSTOM OUTLIER REMOVAL TRANSFORMERS ###
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.exceptions import NotFittedError

## OUTLIER BASE CLASS
class OutlierRemover(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):        
        self.columns = columns  
        self.fitted = False

    def fit_range(self, range): 
        self.range = range
        self.fitted = True
        return self

    def transform(self, X: pd.DataFrame,y=None):
        if not self.fitted: raise NotFittedError()
        filters = reduce(lambda filters, d: filters & self._append_filter(X, d), self._cols(X), True) 
        return X.copy()[filters]
    
    def _append_filter(self, X: pd.DataFrame, f: str) -> bool:
        min, max = self.range[f]
        return (X[f] >= min) & (X[f] <= max)
    
    def _cols(self, X: pd.DataFrame): return self.columns or list(X.columns)

## REMOVE OUTLIERS USING A CUSTOM RANGE (min, max)    
class MinMaxOutlierRemover(OutlierRemover):
    def __init__(self,range=(0,1), columns=None):
        super().__init__(columns)
        self.range = range
    
    def fit(self, X, y=None):
        return super().fit_range({f: self.range for f in self._cols(X)})

## REMOVE OUTLIERS USING IQR
class IQROutlierRemover(OutlierRemover):
    def __init__(self,factor=1.5, columns=None):
        super().__init__(columns)
        self.factor = factor
    
    def _find_range(self, X: pd.DataFrame, f: str):
        Q1 = X[f].quantile(0.25)
        Q3 = X[f].quantile(0.75)
        IQR = Q3 - Q1
        return(Q1 - self.factor * IQR, Q3 + self.factor * IQR)

    def fit(self, X, y=None):
        return super().fit_range({f: self._find_range(X,f) for f in self._cols(X)})

## REMOVE OUTLIERS USING Z-SCORE
class ZScoreOutlierRemover(OutlierRemover):
    def __init__(self, factor=3, columns=None):
        super().__init__(columns)
        self.factor = factor
    
    def _find_range(self, X: pd.DataFrame, f: str):
        mean = X[f].mean()
        sd = X[f].std()
        return (mean - sd * self.factor, mean + sd * self.factor)
    
    def fit(self, X, y=None):
        return super().fit_range({f: self._find_range(X,f) for f in self._cols(X)})

## TRANSFORMATOR TO MERGE AND GROUP VALUES
class TagFeatureLevels(BaseEstimator, TransformerMixin):
    def __init__(self, tags={}, columns=None):        
        self.tags = tags 
        self.columns = columns  
        self.fitted = False

    def _cols(self, X: pd.DataFrame): return self.columns or list(X.columns)

    def fit(self, X, y=None): 
        flatten_entry = lambda k, vals: {v: k for v in vals}
        flat_tags = reduce(lambda a, l: {**a, **flatten_entry(l[0], l[1])}, self.tags.items(), {})
        self._replacemap = {c: flat_tags for c in self._cols(X)}
        self.fitted = True
        return self

    def transform(self,X: pd.DataFrame,y=None):
        if not self.fitted: raise NotFittedError()
        return X.replace(self._replacemap)


## Using the pipeline to clean datasets

In [1658]:
headers = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'ft_income']
df = pd.read_csv('../../datasets/adults.csv', engine='python', sep=', ', names=headers, na_values=['?'])

# df.head(5)
df[["workclass", "occupation", "native-country"]].isna().sum()

workclass         1836
occupation        1843
native-country     583
dtype: int64

In [1659]:
X = df.drop(['ft_income'],axis=1)
y = df['ft_income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42 ) 
steps = []

In [1660]:
steps.extend([
    ('IQR outlier removal', IQROutlierRemover(factor=1.5, columns=["fnlwgt", "hours-per-week"])),
    ('Threshold outlier removal', MinMaxOutlierRemover(range=(0, 30000), columns=["capital-gain"]))
])

### Transform columns

In [1661]:
tf_impute = ["occupation", "native-country"]
tf_merge = ["education"]
tf_scale = ['fnlwgt','capital-gain', 'capital-loss', 'hours-per-week']
tf_pass = ['age', 'workclass', 'marital-status','relationship', 'race', 'sex']

transformers = [
    #Categorical
    ('Impute missing workclass', SimpleImputer(strategy='most_frequent'), tf_impute),
    ('Merge education levels',
        TagFeatureLevels(tags={ 
            "MS-dropout": ['7th-8th','5th-6th','1st-4th'], 
            "HS-dropout": ['9th','10th','11th','12th']
        }),
        tf_merge
    ),
    # Continues
    ('Continues features', StandardScaler(), tf_scale),
    # Skip others  
    ('Keep clean columns','passthrough',tf_pass)
]

order = tf_impute + tf_merge + tf_scale + tf_pass

In [1662]:
# transformer = TagFeatureLevels(
#     tags={ 
#         "MS-dropout": ['7th-8th','5th-6th','1st-4th'], 
#         "HS-dropout": ['9th','10th','11th','12th']
#     }, 
#     columns=["education"]
# )
# X_train = transformer.fit_transform(X_train)

# print(f"before: {X_train.shape[0]} rows")
# transformerA = ZScoreOutlierRemover(factor=3, columns=["fnlwgt", "hours-per-week"])
# X_train = transformerA.fit_transform(X_train)
# transformerB = MinMaxOutlierRemover(range=(0, 30000), columns=["capital-gain"])
# X_train = transformerB.fit_transform(X_train)
# print(f"after: {X_train.shape[0]} rows")

# imputed = pd.DataFrame(X_train, columns=X_train.columns)
# imputed["education"].unique()

In [1663]:
steps.append((
    'Column transformer',
    ColumnTransformer( 
        transformers=transformers, 
        remainder='drop', # education-num
        verbose=True
    )
))

pipeline = Pipeline(steps=steps, verbose=True)

print(f"before: {X_train.shape[0]} rows")
clean_data = pipeline.fit_transform(X_train)
print(f"after: {clean_data.shape[0]} rows")

X_clean = pd.DataFrame(
    clean_data, 
    columns=order
)


before: 26048 rows
[Pipeline]  (step 1 of 3) Processing IQR outlier removal, total=   0.0s
[Pipeline]  (step 2 of 3) Processing Threshold outlier removal, total=   0.0s
[ColumnTransformer]  (1 of 4) Processing Impute missing workclass, total=   0.0s
[ColumnTransformer]  (2 of 4) Processing Merge education levels, total=   0.0s
[ColumnTransformer]  (3 of 4) Processing Continues features, total=   0.0s
[ColumnTransformer]  (4 of 4) Processing Keep clean columns, total=   0.0s
[Pipeline]  (step 3 of 3) Processing Column transformer, total=   0.0s
after: 18199 rows


In [1664]:
def categorial(dataset, features):
    headers = ["feature","count", "% miss.", "card.", "Mode", "Mode freq.", "Mode %", "2nd Mode", "2nd Mode freq.", "2nd Mode %"]
    report = []
    for feature in features:
        report.append([
            feature,
            dataset[feature].size,
            dataset[feature].isnull().sum() / dataset[feature].size,
            dataset[feature].nunique(),
            dataset[feature].mode().values[0],
            dataset[feature].value_counts().max(),
            (dataset[feature].value_counts().max() / dataset[feature].size) * 100,
            
            dataset[feature].value_counts().index[1],
            dataset[feature].value_counts().iloc[1],
            (dataset[feature].value_counts().iloc[1] / dataset[feature].size) * 100,
        ])
    return pd.DataFrame(report, columns=headers)

categorial_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

In [1665]:
categorial_report = X_train.pipe(categorial, categorial_cols)
categorial_report

,feature,count,% miss.,card.,Mode,Mode freq.,Mode %,2nd Mode,2nd Mode freq.,2nd Mode %
0,workclass,26048,0.1,8,Private,18118,69.6,Self-emp-not-inc,2043,7.8
1,education,26048,0.0,16,HS-grad,8416,32.3,Some-college,5806,22.3
2,marital-status,26048,0.0,7,Married-civ-spouse,12026,46.2,Never-married,8557,32.9
3,occupation,26048,0.1,14,Prof-specialty,3312,12.7,Craft-repair,3278,12.6
4,relationship,26048,0.0,6,Husband,10603,40.7,Not-in-family,6603,25.3
5,race,26048,0.0,5,White,22221,85.3,Black,2525,9.7
6,sex,26048,0.0,2,Male,17403,66.8,Female,8645,33.2
7,native-country,26048,0.0,41,United-States,23300,89.5,Mexico,529,2.0


In [1666]:
categorial_report = X_clean.pipe(categorial, categorial_cols)
categorial_report

,feature,count,% miss.,card.,Mode,Mode freq.,Mode %,2nd Mode,2nd Mode freq.,2nd Mode %
0,workclass,18199,0.0,8,Private,13100,72.0,Local-gov,1311,7.2
1,education,18199,0.0,11,HS-grad,6231,34.2,Some-college,3815,21.0
2,marital-status,18199,0.0,7,Married-civ-spouse,8842,48.6,Never-married,5392,29.6
3,occupation,18199,0.0,14,Craft-repair,3399,18.7,Exec-managerial,2361,13.0
4,relationship,18199,0.0,6,Husband,7856,43.2,Not-in-family,4719,25.9
5,race,18199,0.0,5,White,15446,84.9,Black,1804,9.9
6,sex,18199,0.0,2,Male,12455,68.4,Female,5744,31.6
7,native-country,18199,0.0,41,United-States,16570,91.0,Mexico,360,2.0


In [1667]:
continues_features = ['fnlwgt','age','capital-gain', 'capital-loss', 'hours-per-week']
X_train[continues_features].describe()

,fnlwgt,age,capital-gain,capital-loss,hours-per-week
count,26048.0,26048.0,26048.0,26048.0,26048.0
mean,189759.2,38.6,1053.6,88.0,40.4
std,105232.9,13.6,7230.8,403.6,12.3
min,12285.0,17.0,0.0,0.0,1.0
25%,117963.0,28.0,0.0,0.0,40.0
50%,178263.5,37.0,0.0,0.0,40.0
75%,237006.5,48.0,0.0,0.0,45.0
max,1484705.0,90.0,99999.0,4356.0,99.0


In [1668]:
X_clean[continues_features] = X_clean[continues_features].astype(float)
X_clean[continues_features].describe()

,fnlwgt,age,capital-gain,capital-loss,hours-per-week
count,18199.0,18199.0,18199.0,18199.0,18199.0
mean,-0.0,39.0,-0.0,0.0,0.0
std,1.0,12.5,1.0,1.0,1.0
min,-1.9,17.0,-0.2,-0.2,-2.1
25%,-0.7,29.0,-0.2,-0.2,-0.4
50%,-0.0,38.0,-0.2,-0.2,-0.4
75%,0.6,47.0,-0.2,-0.2,0.1
max,2.7,90.0,10.9,10.5,2.6


In [1669]:
print(X_train["education"].unique())
print(X_clean["education"].unique())

['Bachelors' 'Assoc-voc' '9th' 'Some-college' '10th' 'HS-grad'
 'Prof-school' 'Assoc-acdm' '11th' '12th' 'Masters' '7th-8th' 'Doctorate'
 '5th-6th' '1st-4th' 'Preschool']
['Bachelors' 'Assoc-voc' 'HS-dropout' 'Some-college' 'HS-grad'
 'Assoc-acdm' 'MS-dropout' 'Doctorate' 'Prof-school' 'Masters' 'Preschool']


In [1670]:
print(X_train["workclass"].unique())
print(X_clean["workclass"].unique())

['Local-gov' 'Private' 'Self-emp-not-inc' nan 'Federal-gov' 'Self-emp-inc'
 'State-gov' 'Without-pay' 'Never-worked']
['Local-gov' 'Private' 'Self-emp-not-inc' 'Federal-gov' nan 'Self-emp-inc'
 'State-gov' 'Without-pay' 'Never-worked']
